# Evolutionary approach to the Flow Shop problem 

The Problem
-------------------------
Suppose you have N jobs and M machines. Each job consists in M operations on each machine. The i-th operation is executed on the i-th machine and each operation could start only if the previous have already finished. Jobs can be executed in any order. Jobs order could differ depending on machine. The problem is to find optimal solution, i.e. that one with the minimal makespan.

Problem overview
-------------------------
The problem is P-hard only for the case when M = 2. Then it's exactly solved by the Johnson's algorithm. The work is dedicated to the case when M > 2, and, moreover, order of jobs on machines could differ. There are several works on the problem, but most of them solves permutation problem, when order on machines remains the same. Our goal is to find an evolutionary approach to the general flow shop problem. 

Chromosome representation
-------------------------
TBD

In [2]:
from models.job import Job, JobId
from models.schedule import Schedule
from typing import List, MutableMapping

"""
:returns Schedule for the order
:param orders - Orders for each machine
"""
def order_to_schedule(orders: List[List[Job]]) -> Schedule:
    result = Schedule()
    stage_finish_map: MutableMapping[JobId, int] = {}
    makespan = 0
    for (machineId, orderForMachine) in enumerate(orders):
        last_job_finishes = 0
        for job in orderForMachine:
            previous_stage_finished = 0
            if job.job_id in stage_finish_map:
                previous_stage_finished = stage_finish_map[job.job_id]
            start_time = max(last_job_finishes, previous_stage_finished)
            if machineId not in result.table:
                result.table[machineId] = []
            result.table[machineId].append((start_time, job.job_id, job.actions[machineId]))
            last_job_finishes = start_time + job.actions[machineId]
            stage_finish_map[job.job_id] = last_job_finishes
        makespan = max(makespan, last_job_finishes)
    result.makespan = makespan
        
    return result
        

In [3]:
from collections import deque
from typing import List, Deque
from models.job import Job
from functools import cmp_to_key


# Solver for the "F2 || C_max" task introduced in the following article:
# S.M. Johnson. Optimal two-and-three-stage production schedules with set-up times included.
# Naval Research Logistic Quaterly, 1:61–68, 1954
# link: https://www.rand.org/content/dam/rand/pubs/papers/2008/P402.pdf
# param: list of jobs to optimize schedule
# returns: jobs ordered to minimize the makespan. As for F2 an order is equal for both stages, list is 1xN
def f2_cmax_johnson_solver(jobs: List[Job]) -> List[Job]:
    # Validate input
    for job in jobs:
        if len(job.actions) != 2:
            raise ValueError("Johnson solver is able to solve F2||C_max only. "
                             f"There is a job with {len(job.actions)} actions, so it couldn't be executed as F2 job")

    # Sort by minimal stage time
    def jobs_comparator(lhs: Job, rhs: Job) -> int:
        return min(lhs.actions) - min(rhs.actions)

    sorted_jobs = sorted(jobs, key=cmp_to_key(jobs_comparator))
    first_stage_jobs: Deque[Job] = deque()
    second_stage_jobs: Deque[Job] = deque()

    for job in sorted_jobs:
        if job.actions[0] < job.actions[1]:
            first_stage_jobs.append(job)
        else:
            second_stage_jobs.appendleft(job)

    return list(first_stage_jobs + second_stage_jobs)

In [4]:
test1 = f2_cmax_johnson_solver([
            Job(1, [4, 8]),
            Job(2, [3, 3]),
            Job(3, [3, 4]),
            Job(4, [1, 4]),
            Job(5, [8, 7])
        ])

print(order_to_schedule([test1, test1]))

###########################
4333111155555555222        
 44443333111111115555555222
###########################
Makespan: 27
